# The OneMax problem

This is a very simple problem, it is simply to create and sovle for a human. The problem is to create an array of all 1's for a randomly instilaised list. i.e. 0101010 would become 111111. The fitness function is also simple as you simply assign a value of 1 to each one so the array with the highest number of ones is the fittest.

## Setting up



In [0]:
from deap import base
from deap import creator 
from deap import tools

import random
import matplotlib.pyplot as plt

This part just defines all of the constants and parameters that are going to used to control the behavior of the algorithm.

In [0]:
# Problem constants
ONE_MAX_LENGTH = 100

# Genetic Algorithm constants
POPULATION_SIZE = 200 # number of individuals in population
P_CROSSOVER = 0.9 # probability for crossover
P_MUTATION = 0.1 # probability for mutating an individual
MAX_GENERATIONS = 50 # max  number of generations for stopping condition

As we are experminting with this code we would like the same experiment several times and get repeatable result, to do this we add a set seed value. Please not you would not normally do this in a normal run.

In [0]:
RANDOM_SEED = 42 
random.seed(RANDOM_SEED)

Here we create the zeroOrOne function which returns a value of either zero or one as that is what our population will consit of.

In [0]:
toolbox = base.Toolbox()
toolbox.register("zeroOrOne", random.randint,0 ,1)

Now we define the Fitness class, as we only have on objective function here the sum of the digits and our goal is to maximize it, we can call the class FitnessMax with a positive weight of 1.0,

In [0]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

There are conventions in DEAP that are recommend that we stick to, this is to create a class called Individual to represent each of the populations individuals. This is created using the creator tool, in this case a list servers as the best classm with the fitness atribute being the fitness class we defined earlier, FitnessMax:

In [0]:
creator.create("Individual", list, fitness=creator.FitnessMax)

Next we create the individualCreator function, which creates an instance of the Individual class filled up with either zeros or ones randomly. We can do this by modifing the zeroOrOne function we made earlier. This is done by using the initRepeat function which was explained earlier. 


In [0]:
toolbox.register("individualCreator",tools.initRepeat,creator.Individual,toolbox.zeroOrOne,ONE_MAX_LENGTH)

The next step is to define the function that creates the population of the list of individuals that we are going to use.

In [0]:
toolbox.register("populationCreator",tools.initRepeat,list,toolbox.individualCreator)

Now to do the fitness calculation or evaluation using DEAP conventions, we start by defining a function that takes an instance of the Individual class as the parameter and return the fitness for it.

In [0]:
def oneMaxFitness(individual):
  return sum(individual), #return a tuple

Now we need to define the evaluate function as a alias of the oneMaxFitness function to keep with DEAP conventions set.

In [0]:
toolbox.register("evaluate", oneMaxFitness)

Now we need to define the selection, mate, and mutate functions . 

In [0]:
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate",tools.mutFlipBit,indpb=1.0/ONE_MAX_LENGTH)

## Evolving Solution

The algorithm would then be implemented in a main() function that would contain the following code

The first step is to create the initial population that will be used, as well as creating the generationCounter variable that we will use to count the amount of generations created.

In [0]:
population = toolbox.populationCreator(n=POPULATION_SIZE)
generationCounter = 0

We then map the evalutate function to the memebers of population and wrap it around a list so it returns a list of tuples.  

In [0]:
fitnessValue = list(map(toolbox.evaluate,population))
print(fitnessValue)

As the fitnessValue matches the inviduals in the population as one to one due to the mapping, we are able to use the zip() function to combine them and assign the matching fitness tuple to the resepective individual.

In [0]:
for individual, fitnessValue in zip(population,fitnessValue):
  individual.fitness.values = fitnessValue

Now we need to extract the fitness values of the individuals as we only have one single objective this is fairly trivail.

In [0]:
fitnessValues = [individual.fitness.values[0] for individual in population]

We should collect statistics on each generation, these can be whatever you want for this demostation we will recored the max and average

In [0]:
maxFitnessValue = []
meanFitnessValue = []

With all of the above setup we are now able to create the mainloop, so we will start by defining the stopping conditions of the loop, this will be the number of generation and if the best solution has been reached.

In [1]:
while max(fitnessValue) < ONE_MAX_LENGTH and generationCounter < MAX_GENERATIONS:
    generationCounter+=1

NameError: name 'fitnessValue' is not defined

Now we need to apply the selection function, as we already defined the tournament size we only need to send the population and size

In [ ]:
offspring = toolbox.select(population, len(population))

We will now clone the offsping so we can apply the next genetic operations to them instead of the original set

In [ ]:
offspring = list(map(toolbox.clone, offspring))

Crossing over of the offspring is the next step, we will use python slicing to pair every even-indexed item of the offspring list with the one following it. We will then use the random function to test if the probablity of them crossing over surpasses P_CROSSOVER constant set earlier. We will then delete the fitness values of the children as they will potentially ended up having different values.

In [ ]:
for child1, child2 in zip(offspring[::2], offspring[1::2]):
    if random.random() < P_CROSSOVER:
        toolbox.mate(child1,child2)
        del child1.fitness.value
        del child2.fitness.value

Then we need to apply the mutation operator, whilst removing the fitness value of the new offspring

In [ ]:
for mutant in offspring:
    if random.random() < P_MUTATION:
        toolbox.mate(mutant)
        del mutate.fitness.values

For individuals that were not crossed over or mutated remained intact the fitness values will not change, so we do not need to calcualte the fitness values again. For the ones that did we need to calculate the new values, we do this checking if the individual has a valid fitness and then doing as we did before.

In [ ]:
newIndividuals = [ind for ind in offspring if not ind.fitness.valid]
newFitnessValues = list(map(toolbox.evaluate,newIndividuals))
for individual,fitnessValue in zip(newIndividuals,newFitnessValues):
    individual.fitness.value = fitnessValue

Replace the old population with the old one 

In [ ]:
population[:] = offspring

Now we can collect the statistis of this population, and print out the results of that population

In [ ]:
fitnessValues = [ind.fitness.values[0] for ind in population]
maxFitness = max(fitnessValues)
meanFitness = sum(fitnessValues)/ len(population)
maxFitnessValues.append(maxFitness)
meanFitnessValues.append(meanFitness)
print("- Generation {}: Max Fitness = {}, Avg Fitness = {}"
      .format(generationCounter, maxFitness, meanFitness))
best_index = fitnessValues.index(max(fitnessValues))
print("Best Individual = ", *population[best_index], "\n")

Once the stopping conditions are met we can  plot the statistics gathered through matplotlib.

In [ ]:
plt.plot(maxFitnessValues, color='red')
plt.plot(meanFitnessValues, color='green')
plt.xlabel('Generation')
plt.ylabel('Max / Average Fitness')
plt.title('Max and Average fitness over Generations')
plt.show()